### WEB SCRAPING-ASSIGNMENT - 3

### 1. Python program which searches all the product under a particular product from www.amazon.in. 

In [1]:
#Importing the libraries

from bs4 import BeautifulSoup
import pandas as pd
import selenium
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import NoSuchElementException   #importing the exception
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

import re
import time


import warnings
warnings.filterwarnings('ignore')

In [2]:
driver = webdriver.Chrome('chromedriver.exe')

In [3]:
driver.get('http://www.amazon.in/')
delay = 10

In [4]:
#Search input
search_field_input = driver.find_element_by_xpath('//*[@id="twotabsearchtextbox"]')
search_field_input.send_keys(input('Enter the product name: '))

Enter the product name: Guitar


In [5]:
#Clicking the search button
search_button = driver.find_element_by_id('nav-search-submit-button')
search_button.click()

### 2. Scrape the following details of each product listed in first 3 pages of your search results and save it in a data frame and csv. In case if any product has less than 3 pages in search results then scrape all the products available under that product name. Details to be scraped are: "Brand Name", "Name of the Product", "Price", "Return/Exchange", "Expected Delivery", "Availability" and “Product URL”. In case, if any of the details are missing for any of the product then replace it by “-“.

#### Extracting the urls from three pages

In [6]:
#Clicking the next button to get the page url for iteration
next_button = driver.find_element_by_xpath("//a[@class = 's-pagination-item s-pagination-next s-pagination-button s-pagination-separator']")
next_button.click()

In [7]:
#Getting the url for iteration.
curren_url = driver.current_url
keyword = re.compile(r'https://www\.amazon\.in/s\?k=[a-zA-Z0-9]+&page=')
aurl = keyword.finditer(curren_url)
for i in aurl:
    a = i.group(0)
url = []
for page in range (1,4):
    driver.get(str(a+str(page)))
    time.sleep(3)
    url_tags = driver.find_elements_by_xpath("//h2[@class = 'a-size-mini a-spacing-none a-color-base s-line-clamp-4' or @class = 'a-size-mini a-spacing-none a-color-base s-line-clamp-2']/a")
    for ur in url_tags:
        url.append(ur.get_attribute('href'))
len(url)

181

We have scraped urls of every products for the specific search keyword.

In [17]:
#Extracting data for each product
brand_name = []
product_name = []
price_list = []
return_list = []
expected_delivery = []
availability = []
product_url = []

for product in url:
    driver.get(product)
    time.sleep(3)
    
    #Extracting the brand name
    
    try:
        brand_tags = driver.find_element_by_xpath("//h1[@class = 'a-size-large a-spacing-none']/span")
        brand_name.append(brand_tags.text.split()[:1])
    except NoSuchElementException:
        brand_name.append('-')
    
    #Extracting the product name
    try:
        product_tags = driver.find_element_by_xpath("//h1[@class = 'a-size-large a-spacing-none']/span")
        product_name.append(product_tags.text)
    except NoSuchElementException:
        product_name.append('-')
            
    #Extracting the price
    try:
        price_tags = driver.find_element_by_xpath("//span[@id='priceblock_ourprice']")
        price_list.append(price_tags.text)
    except NoSuchElementException:
        price_list.append('-')
            
    #Extracting return/exchange information
    try:
        return_tags = driver.find_element_by_xpath("//div[@data-cel-widget = 'RETURNS_POLICY']/span/div[2]/a")
        return_list.append(return_tags.text)
    except NoSuchElementException:
        return_list.append('-')
            
    #Extracting expected delivery date
    try:
        expected_tags = driver.find_element_by_xpath("//div[@id='ddmDeliveryMessage']/b")
        expected_delivery.append(expected_tags.text)
    except NoSuchElementException:
        expected_delivery.append('-')

    #Extracting Availability
    try:
        availability_tags = driver.find_element_by_xpath("//div[@id='availability']/span")
        availability.append(availability_tags.text)
    except NoSuchElementException:
        availability.append('-')

print(len(brand_name))
print(len(product_name))
print(len(price_list))
print(len(return_list))
print(len(expected_delivery))
print(len(availability))
print(len(url))

181
181
181
181
181
181
181


In [18]:
amazon_products = pd.DataFrame({})

amazon_products['Brand'] = brand_name
amazon_products['Product Name'] = product_name
amazon_products['Price'] = price_list
amazon_products['Retur/Exchange'] = return_list
amazon_products['Expected Delivery'] = expected_delivery
amazon_products['Availability'] = availability
amazon_products['Product URL'] = url

amazon_products.head(10)

,Brand,Product Name,Price,Retur/Exchange,Expected Delivery,Availability,Product URL
0,[Kadence],Kadence Frontier guitar with Online Guitar lea...,-,7 Days Replacement,"Monday, June 6",,https://www.amazon.in/gp/slredirect/picassoRed...
1,[Kadence],Kadence Frontier Jumbo Semi Acoustic Guitar Wi...,-,7 Days Replacement,-,In stock.,https://www.amazon.in/gp/slredirect/picassoRed...
2,[Blueberry],"Blueberry 38C, 38"" Acoustic Guitar Kit with Ba...",-,7 Days Replacement,-,In stock.,https://www.amazon.in/gp/slredirect/picassoRed...
3,[Festra],"Festra 38C, 38 Inch Acoustic Guitar Kit with B...",-,7 Days Replacement,-,In stock.,https://www.amazon.in/gp/slredirect/picassoRed...
4,[Juârez],"Juârez Acoustic Guitar, 38 Inch Cutaway, 038C ...",-,7 Days Replacement,-,In stock.,https://www.amazon.in/Juarez-Acoustic-Cutaway-...
5,[Intern],"Intern INT-38C Acoustic Guitar Kit, With Bag, ...",-,7 Days Replacement,-,In stock.,https://www.amazon.in/Intern-INT-38C-Acoustic-...
6,[Juarez],"Juarez Acoustic Guitar, 38 Inch Curved Body Cu...",-,7 Days Replacement,-,In stock.,https://www.amazon.in/Juarez-Acoustic-Guitar-C...
7,[Intern],"Intern INT-38C Acoustic Guitar Kit, With Bag, ...",-,7 Days Replacement,-,In stock.,https://www.amazon.in/Intern-INT-38C-Acoustic-...
8,[Kadence],Kadence Frontier guitar with Online Guitar lea...,-,7 Days Replacement,"Monday, June 6",,https://www.amazon.in/Kadence-Frontier-Acousti...
9,[Juarez],"Juarez Acoustic Guitar Kit, 38 Inch Cutaway, 3...",-,7 Days Replacement,-,In stock.,https://www.amazon.in/JUAREZ-JRZ38C-Acoustic-S...


In [19]:
#Saving the DataFrame as csv.

amazon_products.to_csv('Amazon Products.csv')

In [20]:
driver.close()

### 3. Python program to access the search bar and search button on images.google.com and scrape 10 images each for keywords ‘fruits’, ‘cars’ and ‘Machine Learning’, ‘Guitar’, ‘Cakes’.

In [201]:
#Importing the libraries

from bs4 import BeautifulSoup
import pandas as pd
import selenium
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import time

import warnings
warnings.filterwarnings('ignore')

In [202]:
driver = webdriver.Chrome('chromedriver.exe')

In [264]:
#User defined function to scrape 10 images from images.google.com for the search keyword
 
def search_google(keyword):  
    driver.get('http://www.images.google.com')
    
    #Accessing the search bar
    input_button = driver.find_element_by_xpath("//input[@class = 'gLFyf gsfi']")
    input_button.send_keys(keyword)
    
    #Clicking the search button

    search_button = driver.find_element_by_xpath("//span[@class = 'z1asCe MZy1Rb']")
    search_button.click()
    
    #Scraping the first 10 image results
    image_tags = driver.find_elements_by_xpath("//div[@class = 'bRMDJf islir']/img")
    images = []
    for image in image_tags:
        images.append(image.get_attribute('src'))
    lt = images[:10]
    
    google_images = pd.DataFrame({keyword:lt})    
    return google_images

In [265]:
search_google('fruits')

,fruits
0,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA..."
1,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA..."
2,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA..."
3,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA..."
4,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA..."
5,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA..."
6,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA..."
7,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA..."
8,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA..."
9,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA..."


In [267]:
search_google('cars')

,cars
0,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA..."
1,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA..."
2,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA..."
3,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA..."
4,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA..."
5,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA..."
6,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA..."
7,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA..."
8,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA..."
9,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA..."


In [268]:
search_google('Machine Learning')

,Machine Learning
0,"data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAA..."
1,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA..."
2,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA..."
3,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA..."
4,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA..."
5,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA..."
6,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA..."
7,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA..."
8,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA..."
9,"data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAA..."


In [269]:
search_google('Guitar')

,Guitar
0,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA..."
1,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA..."
2,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA..."
3,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA..."
4,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA..."
5,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA..."
6,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA..."
7,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA..."
8,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA..."
9,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA..."


In [270]:
search_google('Cakes')

,Cakes
0,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA..."
1,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA..."
2,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA..."
3,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA..."
4,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA..."
5,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA..."
6,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA..."
7,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA..."
8,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA..."
9,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA..."


In [ ]:
driver.close()

### 4. Python program to search for a smartphone(e.g.: Oneplus Nord, pixel 4A, etc.) on www.flipkart.com and scrape following details for all the search results displayed on 1st page.

Details to be scraped: 

“Brand Name”, “Smartphone name”, “Colour”, “RAM”, “Storage(ROM)”, “Primary Camera”,
“Secondary Camera”, “Display Size”, “Battery Capacity”, “Price”, “Product URL”. Incase if any of the
details is missing then replace it by “- “. Save your results in a dataframe and CSV.

In [1]:
#Importing the libraries

from bs4 import BeautifulSoup
import pandas as pd
import selenium
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import time
from selenium.common.exceptions import NoSuchElementException
import numpy as np

from selenium.webdriver.chrome.options import Options     #importing Options
chrome_options = Options()
chrome_options.add_argument('--incognito')

import warnings
warnings.filterwarnings('ignore')

In [278]:
driver = webdriver.Chrome('chromedriver.exe')

In [288]:
driver.get('http://www.flipkart.com/')

#Closing the login pop-up

try:
    
    login_popup = driver.find_element_by_xpath("//button[@class = '_2KpZ6l _2doB4z']")
    login_popup.click()

except NoSuchElementException as ne:
    print("Exception Raised", '(NoSuchElement)')

Exception Raised (NoSuchElement)


In [327]:
#User defined function to scrape details for smartphone from www.flipkart.com

def smartphone(phone):
    driver.get('http://www.flipkart.com/')

    #Closing the login pop-up

    try:
        login_popup = driver.find_element_by_xpath("//button[@class = '_2KpZ6l _2doB4z']")
        login_popup.click()
    except NoSuchElementException as ne:
        print("Exception Raised", '(No popup to close)')
        
        
    #Inputting the keyword   
    input_keyword = driver.find_element_by_xpath("//input[@class = '_3704LK']")
    input_keyword.send_keys(phone+" smartphone")

    #Searching

    search_button = driver.find_element_by_xpath("//button[@class = 'L0Z3Pu']")
    search_button.click()
    
    brand_name = []
    smartphone_name = []
    colour_list = []
    RAM = []
    storage = []
    primary_camera = []
    secondary_camera = []
    display_size = []
    battery = []
    price  = []
    product_url = []

    time.sleep(2)
    brand_tags = driver.find_elements_by_xpath("//div[@class ='_4rR01T']")

    for brand in brand_tags:
        brand_name.append(brand.text.split()[0])
    time.sleep(1)
    print(len(brand_name))
    

    smartphone_tags = driver.find_elements_by_xpath("//div[@class ='_4rR01T']")

    for smart in smartphone_tags:
        smartphone_name.append(smart.text)
    time.sleep(1)
    print(len(smartphone_name))

    colour_tags = driver.find_elements_by_xpath("//div[@class ='_4rR01T']")

    for colour in colour_tags:
        colour_list.append(colour.text.split()[-4])
    time.sleep(1)
    print(len(colour_list))

    ram_tags = driver.find_elements_by_xpath("//div[@class ='fMghEO']/ul/li[1]")

    for ram in ram_tags:
        RAM.append(ram.text[:9])
    time.sleep(1)
    print(len(RAM))

    storage_tags = driver.find_elements_by_xpath("//div[@class ='fMghEO']/ul/li[1]")

    for store in storage_tags:
        storage.append(store.text[-10:])
    time.sleep(1)
    print(len(storage))

    primary_tags = driver.find_elements_by_xpath("//div[@class ='fMghEO']/ul/li[3]")
    for primary in primary_tags:
        primary_camera.append(primary.text[:17])
    time.sleep(1)
    print(len(primary_camera))

    secondary_tags = driver.find_elements_by_xpath("//div[@class ='fMghEO']/ul/li[3]")

    for secondary in secondary_tags:
        secondary_camera.append(secondary.text[19:])
    for i in range (0,len(secondary_camera)):
        if secondary_camera[i] == '':
            secondary_camera[i] = '-'
    time.sleep(1)
    print(len(secondary_camera))

    display_tags = driver.find_elements_by_xpath("//div[@class ='fMghEO']/ul/li[2]")

    for display in display_tags:
        display_size.append(display.text)
    time.sleep(1)
    print(len(display_size))

    battery_tags = driver.find_elements_by_xpath("//div[@class ='fMghEO']/ul/li[4]")
    for bat in battery_tags:
        battery.append(bat.text)
    time.sleep(1)
    print(len(battery))

    price_tags = driver.find_elements_by_xpath("//div[@class = '_30jeq3 _1_WHN1']")

    for pr in price_tags:
        price.append(pr.text)
    time.sleep(1)
    print(len(price))

    url_tags = driver.find_elements_by_xpath("//a[@class = '_1fQZEK']")

    for url in url_tags:
        product_url.append(url.get_attribute('href'))
    time.sleep(1)
    print(len(product_url))
    
    
    
    smartphone_data = pd.DataFrame({})
    smartphone_data['Brand '] = brand_name
    smartphone_data['Smartphone'] = smartphone_name
    smartphone_data['Colour'] = colour_list
    smartphone_data['RAM'] = RAM 
    smartphone_data['ROM(Storage)'] = storage
    smartphone_data['Primary Camera'] = primary_camera 
    smartphone_data['Secondary Camera'] = secondary_camera
    smartphone_data['Display Size'] = display_size 
    smartphone_data['Battery Capacity'] = battery
    smartphone_data['Price'] = price
    smartphone_data['Product URL'] = product_url
    smartphone_data.replace(r'^\s*$', np.nan, regex=True)
    
    return smartphone_data
    


In [330]:
Oneplus = smartphone('Oneplus Nord')
Oneplus.to_csv('Flipkart-oneplusnord.csv')

Exception Raised (No popup to close)
24
24
24
24
24
24
24
24
24
24
24


In [331]:
Oneplus

,Brand,Smartphone,Colour,RAM,ROM(Storage),Primary Camera,Secondary Camera,Display Size,Battery Capacity,Price,Product URL
0,OnePlus,"OnePlus Nord CE 2 Lite 5G (Black Dusk, 128 GB)",(Black,6 GB RAM,128 GB ROM,64MP Rear Camera,-,16.74 cm (6.59 inch) Display,5000 mAh Battery,"₹19,930",https://www.flipkart.com/oneplus-nord-ce-2-lit...
1,OnePlus,"OnePlus Nord 2 5G (Blue Haze, 128 GB)",(Blue,8 GB RAM,128 GB ROM,50MP Rear Camera,-,16.33 cm (6.43 inch) Display,4500 mAh Battery,"₹29,535",https://www.flipkart.com/oneplus-nord-2-5g-blu...
2,OnePlus,"OnePlus Nord 2 5G (PAC-MAN, 256 GB)",5G,12 GB RAM,256 GB ROM,50MP Rear Camera,-,16.33 cm (6.43 inch) Display,4500 mAh Battery,"₹37,890",https://www.flipkart.com/oneplus-nord-2-5g-pac...
3,OnePlus,"OnePlus Nord 2 5G (Gray Sierra, 128 GB)",(Gray,8 GB RAM,128 GB ROM,50MP Rear Camera,-,16.33 cm (6.43 inch) Display,4500 mAh Battery,"₹29,141",https://www.flipkart.com/oneplus-nord-2-5g-gra...
4,OnePlus,"OnePlus Nord CE 2 Lite 5G (Black Dusk, 128 GB)",(Black,8 GB RAM,128 GB ROM,64MP Rear Camera,-,16.74 cm (6.59 inch) Display,5000 mAh Battery,"₹21,997",https://www.flipkart.com/oneplus-nord-ce-2-lit...
5,OnePlus,"OnePlus Nord CE 2 5G (Gray Mirror, 128 GB)",(Gray,8 GB RAM,128 GB ROM,64MP Rear Camera,-,16.33 cm (6.43 inch) Display,4500 mAh Battery,"₹24,999",https://www.flipkart.com/oneplus-nord-ce-2-5g-...
6,OnePlus,"OnePlus Nord CE 2 5G (Bahama Blue, 128 GB)",(Bahama,8 GB RAM,128 GB ROM,64MP Rear Camera,-,16.33 cm (6.43 inch) Display,4500 mAh Battery,"₹24,998",https://www.flipkart.com/oneplus-nord-ce-2-5g-...
7,OnePlus,"OnePlus Nord (Blue Marble, 128 GB)",(Blue,8 GB RAM,128 GB ROM,48MP + 8MP | 32MP,8MP Dual Front Camera,16.36 cm (6.44 inch) Full HD+ Display,4115 mAh Battery,"₹27,999",https://www.flipkart.com/oneplus-nord-blue-mar...
8,OnePlus,"OnePlus Nord (Gray Onyx, 256 GB)",(Gray,12 GB RAM,256 GB ROM,48MP + 8MP | 32MP,8MP Dual Front Camera,16.36 cm (6.44 inch) Full HD+ Display,4115 mAh Battery,"₹29,999",https://www.flipkart.com/oneplus-nord-gray-ony...
9,OnePlus,"OnePlus Nord (Gray Onyx, 64 GB)",(Gray,6 GB RAM,64 GB ROM,48MP + 8MP | 32MP,8MP Dual Front Camera,16.36 cm (6.44 inch) Full HD+ Display,4115 mAh Battery,"₹24,999",https://www.flipkart.com/oneplus-nord-gray-ony...


In [335]:
driver.close()

### 5. Python program to scrap geospatial coordinates (latitude, longitude) of a city searched on google maps.

In [34]:
#Importing the libraries

from bs4 import BeautifulSoup
import pandas as pd
import selenium
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import time
from selenium.common.exceptions import NoSuchElementException
import re

from selenium.webdriver.chrome.options import Options     #importing Options
chrome_options = Options()
chrome_options.add_argument('--incognito')

import warnings
warnings.filterwarnings('ignore')

In [137]:
driver = webdriver.Chrome('chromedriver.exe')

In [376]:
driver.get('https://www.google.com/maps')

In [377]:
#User defined function to scrape the geospatial cordinates of a searched city.
def geo_coordinates(city):
    global c
    c = city
    driver.get('https://www.google.com/maps')
    #Inputting the city name
    search_input = driver.find_element_by_xpath("//input[@autofocus = 'autofocus']")
    search_input.send_keys(city)

    #Searching

    search_button = driver.find_element_by_xpath("//div[@class = 'pzfvzf']")
    search_button.click()
    time.sleep(3)
    return (driver)

In [378]:
def geo_code(geo):
    gurl = driver.current_url
    keyword = re.compile(r'https://www\.google\.com/maps/place/[a-zA-Z0-9\+,]+/@([0-9\.]+,[0-9\.-]+)')
    aurl = keyword.finditer(str(gurl))
    for i in aurl:
        stra = str(i.group(1))
        print("Geospacial Coordinates of", c," is",stra)


In [380]:
geo = geo_coordinates('New York')
geo_code(geo)

Geospacial Coordinates of New York  is 40.6976701,-74.2598789


In [381]:
driver.close()

### 6. Python program to scrap details of all the funding deals for second quarter (i.e Jan 21 – March 21) from trak.in

In [482]:
#Importing the libraries

from bs4 import BeautifulSoup
import pandas as pd
import selenium
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import time
from selenium.common.exceptions import ElementNotInteractableException
from selenium.webdriver.common.action_chains import ActionChains

from selenium.webdriver.chrome.options import Options     #importing Options
chrome_options = Options()
chrome_options.add_argument('--incognito')

import warnings
warnings.filterwarnings('ignore')

In [486]:
driver = webdriver.Chrome('chromedriver.exe')
driver.maximize_window() 

In [487]:
driver.get('https://trak.in/')

In [488]:
#Clicing the category - Funding deals

try:
    more_button = driver.find_element_by_css_selector("#main-navigation > li.bs-pretty-tabs-container.menu-item-has-children.menu-item.better-anim-fade.bsm-leave > a").click()
    funding_deal_button = driver.find_element_by_css_selector("#menu-item-51510 > a").click()
except ElementNotInteractableException as ie:
    print("Exception Raised")
    funding_deal_button = driver.find_element_by_css_selector("#menu-item-51510 > a").click()

Exception Raised


In [509]:
#Extracting details
date = []
startup = []
industry = []
sub_vertical = []
location = []
investor = []
investment = []
amount = []

for i in range(54,57):
    #Scraping date details
    date_tags = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[2]'.format(i))
    for dt in date_tags:
        date.append(dt.text)

    #Scraping start up details
    startup_tags= driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[3]'.format(i))
    for start in startup_tags:
        startup.append(start.text)
    
    #Scraping industry details
    industry_tags = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[4]'.format(i))
    for ind in industry_tags:
        industry.append(ind.text)
        
    #Scraping  sub vertical details
    sub_tags = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[5]'.format(i))
    for sub in sub_tags:
        sub_vertical.append(sub.text)
        
    #Scraping  location details
    location_tags = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[6]'.format(i))
    for loc in location_tags:
        location.append(loc.text)
    
    #Scraping  investor details
    investor_tags = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[7]'.format(i))
    for inv in investor_tags:
        investor.append(inv.text)
    
    #Scraping  investment details
    investment_tags = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[8]'.format(i))
    for invest in investment_tags:
        investment.append(invest.text)
    
    #Scraping  amount
    amount_tags = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[9]'.format(i))
    for am in amount_tags:
        amount.append(am.text)
    
    
print(len(date))
print(len(startup))
print(len(industry))
print(len(sub_vertical))
print(len(location))
print(len(investor))
print(len(investment))
print(len(amount))

24
24
24
24
24
24
24
24


In [510]:
funding_deals = pd.DataFrame({})
funding_deals['Date'] = date
funding_deals['Startup Name'] = startup
funding_deals['Industry/Vertical'] = industry
funding_deals['Sub-Vertical'] =  sub_vertical
funding_deals['City/Location'] = location
funding_deals["Investors' Name"] = investor
funding_deals['Investment Type'] = investment
funding_deals['Amount(In USD)'] = amount

funding_deals

,Date,Startup Name,Industry/Vertical,Sub-Vertical,City/Location,Investors' Name,Investment Type,Amount(In USD)
0,15/01/2021,Digit Insurance,Financial Services,Insurance Services,Bengaluru,"A91 Partners, Faering Capital, TVS Capital Funds",Venture,"1,80,00,000"
1,28/01/2021,Bombay Shaving Company,Consumer Goods Company,"Shave care, beard care, and skincare products",New Delhi,Reckitt Benckiser,Venture,"6,172,258.50"
2,19/01/2021,DeHaat,AgriTech Startup,online marketplace for farm products and services,Patna,Prosus Ventures,Series C,"30,000,000"
3,19/01/2021,Darwinbox,SaaS,HR Tech,Mumbai,Salesforce Ventures,Seed,"15,000,000"
4,18/01/2021,mfine,Health Tech Startup,AI-powered telemedicine mobile app,Bengaluru,Heritas Capital Management,Venture Round,"16,000,000"
5,18/01/2021,Udayy,EdTech,Online learning platform for kids in class 1-5,Gurgaon,Sequoia Capital,Seed Funding,"15,000,000"
6,11/01/2021,True Elements,Food Startup,Whole Food plant based Nashta,Pune,SIDBI Venture Capital,Series,"100,000,000"
7,13/01/2021,Saveo,B2B E-commerce,Pharmacies,Bengaluru,"Matrix Partners India, RTP Global, others",Seed,"4,000,000"
8,11/02/2021,Doubtnut,Edu Tech,E-Learning Platform,Gurgaon,"SIG Global, Sequoia Capital, WaterBridge Ventu...",Series B,"2,500,000"
9,22/02/2021,Zomato,Hospitality,Online Food Delivery Platform,Gurgaon,"Tiger Global, Kora",Venture,"250,000,000"


In [511]:
driver.quit()

### 7. Python program to scrap all the available details of best gaming laptops from digit.in.

In [382]:
#Importing the libraries

from bs4 import BeautifulSoup
import pandas as pd
import selenium
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import time
from selenium.common.exceptions import ElementNotInteractableException
from selenium.webdriver.common.action_chains import ActionChains

from selenium.webdriver.chrome.options import Options     #importing Options
chrome_options = Options()
chrome_options.add_argument('--incognito')

import warnings
warnings.filterwarnings('ignore')
from selenium.webdriver.common.keys import Keys

In [383]:
driver = webdriver.Chrome('chromedriver.exe')

In [456]:
driver.get("https://www.digit.in/")

In [457]:
#Clicking the search button to show the search field
search_button = driver.find_element_by_css_selector("body > div.header > div > div:nth-child(3) > div.search > a > img").click()
search_input = driver.find_element_by_css_selector("#globalPageSearchText").send_keys('best gaming laptops',Keys.RETURN)

In [458]:
#Extracting the urls for products from first page.
url = []

url_tags = driver.find_elements_by_xpath("//div[@class = 'searchPage']/a")
for ur in url_tags:
    url.append(ur.get_attribute('href'))
print(len(url))

10


In [459]:
##Extracting the details 
brand_name = []
product_name = []
processor_name = []
os_name = []
display = []
memory = []
price = []
release = []
details = []

for i in url:
    driver.get(i)
    time.sleep(3)
    
    #Getting the brand name
    try:
        brand_name.append(driver.find_element_by_xpath("//div[@class = 'status_bar']/div[3]/strong").text)
    except NoSuchElementException:
        brand_name.append("-")
  
    #Getting the product name
    try:
        product_name.append(driver.find_element_by_xpath("//div[@class = 'heading-wraper']/h1").text)
    except NoSuchElementException:
        product_name.append("-")
   
    #Getting the processor details
    try:
        processor_name.append(driver.find_element_by_xpath("//div[@class = 'specs_details ']/ul/li[3]/div/p/strong").text)
    except NoSuchElementException:
        processor_name.append("-")
        
    #Getting the OS details
    try:
        os_name.append(driver.find_element_by_xpath("//div[@class = 'specs_details ']/ul/li[1]/div/p/strong").text)
    except NoSuchElementException:
        os_name.append("-")
  
    #Getting the display details
    try:
        display.append(driver.find_element_by_xpath("//div[@class = 'specs_details ']/ul/li[2]/div/p/strong").text)
    except NoSuchElementException:
        display.append("-")
    
    #Getting the memory details
    try:
        memory.append(driver.find_element_by_xpath("//div[@class = 'specs_details ']/ul/li[4]/div/p/strong").text)
    except NoSuchElementException:
        memory.append("-")

    #Getting the price details
    try:
        price.append(driver.find_element_by_xpath("//div[@class = 'price_sec']/div/h2/strong").text)
    except NoSuchElementException:
        price.append("-")

    #Getting the release date
    try:
        release.append(driver.find_element_by_xpath("//div[@class = 'status_bar']/div[2]/strong").text)
    except NoSuchElementException:
        release.append("-")

    #Getting the Other details
    try:
        details.append(driver.find_element_by_xpath("//div[@class = 'common_container']/div[2]/p").text)
    except NoSuchElementException:
        details.append("-")
        
print(len(brand_name))        
print(len(product_name))
print(len(processor_name))
print(len(os_name))
print(len(display))
print(len(memory))
print(len(price))
print(len(release))
print(len(details))
    
gaming_laptops = pd.DataFrame({})
gaming_laptops['Brand'] = brand_name
gaming_laptops['Laptop Name'] = product_name
gaming_laptops['Processor'] = processor_name
gaming_laptops['OS'] = os_name
gaming_laptops['Display Size'] = display
gaming_laptops['Memory'] = memory
gaming_laptops['Price'] = price
gaming_laptops['Release Date'] = release
gaming_laptops['Availability in India'] = details

    
gaming_laptops

10
10
10
10
10
10
10
10
10


,Brand,Laptop Name,Processor,OS,Display Size,Memory,Price,Release Date,Availability in India
0,Asus,Asus G750JX-CV069P,NA,NA,NA,-,"₹ 149,999","14 Oct, 2014",Asus G750JX-CV069P price in India starts from ...
1,Lenovo,Lenovo G50-30,NA,NA,NA,-,"₹ 25,694","17 Mar, 2016",Lenovo G50-30 price in India starts from ₹ 256...
2,Asus,ASUS G53J - A 3D gamer's delight but we want more,NA,NA,NA,-,"₹ 112,499","13 Jan, 2011",-
3,Dell,Dell Vostro 15 3500,11th Gen Intel® Core™ I3-1115G4 | NA,Windows 10 Home,"15.6"" (1920 X 1080)",1 TB SATA/8 GBGB DDR4,"₹ 38,489","17 Mar, 2021",-
4,HP,HP Envy dv6-7206tx notebook,Intel Core I7 (3rd Generation) | 2.4 Ghz,Windows 8 64 Bit,"15.6"" (1366 X 768)",1 TB SATA/8GB DDR3,"₹ 65,705","23 Feb, 2013",-
5,HP,HP Envy 15-k006tx,NA,NA,NA,-,"₹ 80,510","14 Oct, 2014",-
6,Asus,ASUS ROG Flow x13,AMD Ryzen™ 9 5980HS Processor 3.1 GHz | NA,Windows 10 Pro,"13.4"" (3840 X 2400)",16GB*2 LPDDR4X/4 GBGB GDDR6,"₹ 154,990","05 May, 2021",ASUS ROG Flow x13 price in India starts from ₹...
7,Samsung,Samsung Series 9 (NP-900X3C),NA,NA,NA,-,-,"01 Nov, 2012",-
8,Asus,Asus GTX980 Matrix Platinum,-,-,-,-,"₹ 52,000","30 Jan, 2015",-
9,Sony,Sony PlayStation Vita,-,-,-,-,"₹ 24,990","18 Feb, 2012",-


In [460]:
driver.close()

### 8. Python program to scrape the details for all billionaires from www.forbes.com.

Details to be
scrapped: “Rank”, “Name”, “Net worth”, “Age”, “Citizenship”, “Source”, “Industry”.

In [133]:
#Importing the libraries

from bs4 import BeautifulSoup
import pandas as pd
import selenium
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import time
from selenium.common.exceptions import ElementNotInteractableException
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.action_chains import ActionChains

from selenium.webdriver.chrome.options import Options     #importing Options
chrome_options = Options()
chrome_options.add_argument('--incognito')

import warnings
warnings.filterwarnings('ignore')
from selenium.webdriver.common.keys import Keys

In [192]:
driver = webdriver.Chrome('chromedriver.exe')

In [193]:
driver.get('http://www.forbes.com/')

In [194]:
driver.find_element_by_xpath("//button[@class = 'fto-main__scroll-down-btn']").click()

In [195]:
#clicking the menu button
action = ActionChains(driver)
menu_button = driver.find_element_by_xpath("//button[@class = 'icon--hamburger']").click()
#clicking the category billionaires
billioinaire_button = driver.find_element_by_xpath("//a[@class = 'header__title']")
action.move_to_element(billioinaire_button).perform()
worlds_button = driver.find_element_by_xpath("//a[@class = 'section__link']").click()

In [197]:
#Extracting the data from first page


rank_list = []
rank_tags = driver.find_elements_by_xpath("//div[@class = 'table-row-group__container']/div/div[1]")

for ran in rank_tags:
    rank_list.append(ran.text)
rank_list.pop(1)
rank_list.pop(-1)
print(len(rank_list))

name_list = []
name_tags = driver.find_elements_by_xpath("//div[@class = 'table-row-group__container']/div/div[2]")
for name in name_tags:
    name_list.append(name.text)
name_list.pop(1)
name_list.pop(-1)
print(len(name_list))

net_worth = []
net_tags = driver.find_elements_by_xpath("//div[@class = 'table-row-group__container']/div/div[3]")

for net in net_tags:
    net_worth.append(net.text)
net_worth.pop(1)
print(len(net_worth))

age_list = []
age_tags = driver.find_elements_by_xpath("//div[@class = 'table-row-group__container']/div/div[4]")

for age in age_tags:
    age_list.append(age.text)
print(len(age_list))

citizenship = []
citizen_tags = driver.find_elements_by_xpath("//div[@class = 'table-row-group__container']/div/div[5]")

for cit in citizen_tags:
    citizenship.append(cit.text)
print(len(citizenship))

source_list = []
source_tags = driver.find_elements_by_xpath("//div[@class = 'table-row-group__container']/div/div[6]")

for source in source_tags:
    source_list.append(source.text)
print(len(source_list))

industry_list = []
industry_tags = driver.find_elements_by_xpath("//div[@class = 'table-row-group__container']/div/div[7]")

for ind in industry_tags:
    industry_list.append(ind.text)
print(len(industry_list))

200
200
200
200
200
200
200


In [202]:
try:
    driver.find_element_by_xpath("//div[@class = 'tp-iframe-wrapper tp-active']/button").click()
except (ElementNotInteractableException or NoSuchElementException):
    print("Exception Raised")

In [204]:
#Extracting details from second page

for i in range (2,8):
    #Clicking the next page button
    next_button = driver.find_element_by_xpath("//div[@class = 'next-page']").click()
    time.sleep(3)
    
    rank_tags = driver.find_elements_by_xpath("//div[@class = 'table-row-group__container']/div/div[1]")

    for ran in rank_tags:
        rank_list.append(ran.text)
    rank_list.pop(1)
    rank_list.pop(-1)
    

    name_tags = driver.find_elements_by_xpath("//div[@class = 'table-row-group__container']/div/div[2]")

    for name in name_tags:
        name_list.append(name.text)
    name_list.pop(1)
    name_list.pop(-1)
    


    net_tags = driver.find_elements_by_xpath("//div[@class = 'table-row-group__container']/div/div[3]")

    for net in net_tags:
        net_worth.append(net.text)
    net_worth.pop(1)
    


    age_tags = driver.find_elements_by_xpath("//div[@class = 'table-row-group__container']/div/div[4]")

    for age in age_tags:
        age_list.append(age.text)
    


    citizen_tags = driver.find_elements_by_xpath("//div[@class = 'table-row-group__container']/div/div[5]")

    for cit in citizen_tags:
        citizenship.append(cit.text)
    


    source_tags = driver.find_elements_by_xpath("//div[@class = 'table-row-group__container']/div/div[6]")

    for source in source_tags:
        source_list.append(source.text)
    


    industry_tags = driver.find_elements_by_xpath("//div[@class = 'table-row-group__container']/div/div[7]")

    for ind in industry_tags:
        industry_list.append(ind.text)
    
    
print(len(rank_list))
print(len(name_list))
print(len(net_worth))
print(len(age_list))
print(len(citizenship))
print(len(source_list))
print(len(industry_list))

1400
1400
1400
1400
1400
1400
1400


In [205]:
#We have scraped all first 1400 billionaire details from the forbes. website.
billionaires = pd.DataFrame({})
billionaires['Rank'] = rank_list
billionaires['Name'] = name_list
billionaires['Net Worth'] = net_worth
billionaires['Age'] = age_list
billionaires['Citizenship'] = citizenship
billionaires['Source'] = source_list
billionaires['Industry'] = industry_list

billionaires

,Rank,Name,Net Worth,Age,Citizenship,Source,Industry
0,1.,Elon Musk,$219 B,50,United States,"Tesla, SpaceX",Automotive
1,8.,Larry Ellison,$106 B,58,United States,Amazon,Technology
2,9.,Steve Ballmer,$91.4 B,73,France,LVMH,Fashion & Retail
3,10.,Mukesh Ambani,$90.7 B,66,United States,Microsoft,Technology
4,11.,Gautam Adani & family,$90 B,91,United States,Berkshire Hathaway,Finance & Investments
...,...,...,...,...,...,...,...
1395,1341.,Xiaojuan Zhang,$2.3 B,52,China,logistics,Logistics
1396,1397.,Michael Ashcroft,$2.2 B,76,United Kingdom,security,Finance & Investments
1397,1397.,Alex Atallah,$2.2 B,30,United States,online marketplace,Technology
1398,1397.,Brett Blundy,$2.2 B,62,Australia,"retail, agribusiness",Fashion & Retail


### 9. Python program to extract at least 500 Comments, Comment upvote and time when comment was posted from any YouTube Video

In [206]:
#Importing the libraries

from bs4 import BeautifulSoup
import pandas as pd
import selenium
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import time
from selenium.common.exceptions import ElementNotInteractableException
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.action_chains import ActionChains

from selenium.webdriver.chrome.options import Options     #importing Options
chrome_options = Options()
chrome_options.add_argument('--incognito')

import warnings
warnings.filterwarnings('ignore')
from selenium.webdriver.common.keys import Keys

In [207]:
driver = webdriver.Chrome('chromedriver.exe')

In [219]:
driver.get('https://www.youtube.com/')

In [220]:
#Clicking the search bar
video = 'Avatar 2'
driver.find_element_by_xpath("//input[@id = 'search']").send_keys(video)
driver.find_element_by_xpath("//button[@id = 'search-icon-legacy']").click()

In [221]:
#Selecting the first video
first_video = driver.find_element_by_xpath("//div[@class = 'text-wrapper style-scope ytd-video-renderer']").click()

In [230]:
#clicking the comment section
comments = driver.find_element_by_xpath("/html/body/ytd-app/div[1]/ytd-page-manager/ytd-watch-flexy/div[5]/div[1]/div/ytd-watch-metadata/div/div[3]/div[2]/ytd-comments-entry-point-header-renderer/div[1]/div[1]").click()

In [248]:
#Extracting the first 500 comments, upvote and time when comment was posted.
for i in range(1,5):

    comments_list = []

    comment_tags = driver.find_elements_by_css_selector("#content-text")
    for com in comment_tags:
        comments_list.append(com.text)
    

    upvotes_list = []
    upvotes_tags = driver.find_elements_by_css_selector("#vote-count-middle")
    for upvote in upvotes_tags:
        upvotes_list.append(upvote.text)
    

    time_posted = []
    time_tags = driver.find_elements_by_css_selector("#header-author > yt-formatted-string > a")
    for time in time_tags:
        time_posted.append(time.text)


    while(True):
        height = driver.execute_script("return document.body.scrollHeight")
        driver.find_element_by_tag_name('body').send_keys(Keys.END)
        if int(height)==0:
            break
    
print(len(comments_list))
print(len(upvotes_list))
print(len(time_posted))

559
579
579


In [250]:
#Creating the dataframe
youtube_comments = pd.DataFrame({})
youtube_comments['Comment'] = comments_list[:500]
youtube_comments['Upvotes'] = upvotes_list[:500]
youtube_comments['Time Posted'] = time_posted[:500]

youtube_comments

,Comment,Upvotes,Time Posted
0,One thing for sure: There will be TONS of gorg...,8.7K,2 weeks ago (edited)
1,Can't believe it was 13 years ago when the fir...,3.9K,2 weeks ago
2,The Avatar series has some of the best visuals...,1.7K,11 days ago
3,It's just so comforting to hear that so many p...,3.2K,11 days ago
4,"When people ask me why I LOVE Avatar I’m like,...",19,16 hours ago
...,...,...,...
495,Hell of a trailer,,10 days ago
496,Thank God for life. So glad that I am alive to...,,3 days ago
497,Very special movie,1,1 day ago
498,"I am sure, this upcoming movie of James Camero...",70,2 weeks ago


In [251]:
driver.close()

### 10. Python program to scrape a data for all available Hostels from https://www.hostelworld.com/ in “London” location

You have to scrape hostel name, distance from city centre, ratings, total reviews,
overall reviews, privates from price, dorms from price, facilities and property description.

In [461]:
#Importing the libraries

from bs4 import BeautifulSoup
import pandas as pd
import selenium
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import time
from selenium.common.exceptions import ElementNotInteractableException
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.action_chains import ActionChains

from selenium.webdriver.chrome.options import Options     #importing Options
chrome_options = Options()
chrome_options.add_argument('--incognito')

import warnings
warnings.filterwarnings('ignore')
from selenium.webdriver.common.keys import Keys

In [470]:
driver = webdriver.Chrome('chromedriver.exe')

In [477]:
driver.get('https://www.hostelworld.com/')

In [478]:
#Searching the keyword
search_input_click = driver.find_element_by_css_selector("#location-text-input-field").click()

In [479]:
search_input = driver.find_element_by_xpath("//input[@id = 'search-input-field']").send_keys('London')

In [480]:
search_button = driver.find_element_by_css_selector("#predicted-search-results > li:nth-child(2) > div").click()

In [481]:
search_button_click = driver.find_element_by_xpath("//button[@id = 'search-button']").click()

In [76]:
#Scraping the data

hostel_name = []
distance = []
ratings = []
total_review = []
overall_review = []
private_price = []
dorms_price = []
facilities_list = []
propoerty_desc = []


hostel_tags = driver.find_elements_by_css_selector("div.title-row > h2 > a")
for host in hostel_tags:
    hostel_name.append(host.text)
print(len(hostel_name))

distance_tags = driver.find_elements_by_xpath("//div[@data-v-38030316]/div[@class = 'subtitle body-3']")
for dist in distance_tags:
    distance.append(dist.text)
print(len(distance))

rating_tags = driver.find_elements_by_xpath("//div[@class = 'rating rating-summary-container big']/div[1]")
for rating in rating_tags:
    ratings.append(rating.text[:9])
print(len(ratings))

review_tags = driver.find_elements_by_xpath("//div[@class = 'rating rating-summary-container big']/div/div[2]")
for review in review_tags:
    total_review.append(review.text)
print(len(total_review))

overall_tags = driver.find_elements_by_xpath("//div[@class = 'rating rating-summary-container big']/div/div[1]")
for over in overall_tags:
    overall_review.append(over.text)
print(len(overall_review))

private_tags = driver.find_elements_by_xpath("//div[@class = 'prices-col']/a/div[1]")
for private in private_tags:
    private_price.append(private.text.replace("\n"," "))
print(len(private_price))

dorms_tags = driver.find_elements_by_xpath("//div[@class = 'prices-col']/a/div[2]")
for dorm in dorms_tags:
    dorms_price.append(dorm.text.replace("\n"," "))
print(len(dorms_price))

facility_tags = driver.find_elements_by_xpath("//div[@class = 'facilities-label facilities']")
for f in facility_tags:
    facilities_list.append(f.text.replace("\n",","))
print(len(facilities_list))

property_tags = driver.find_elements_by_xpath("//div[@class = 'rating-factors prop-card-tablet rating-factors small']")
for prop in property_tags:
    propoerty_desc.append(prop.text.replace("\n",","))
print(len(propoerty_desc))




hostel_world = pd.DataFrame({})
hostel_world['Hostel Name'] = hostel_name
hostel_world['Distance from City Centre'] = distance
hostel_world['Ratings'] = ratings
hostel_world['Total Reviews'] = total_review
hostel_world['Overall Review'] = overall_review
hostel_world['Privates from Price'] = private_price
hostel_world['Dorms from Price'] = dorms_price
hostel_world['Facilities'] = facilities_list
hostel_world['Property Description'] = propoerty_desc

hostel_world

25
25
25
25
25
25
25
25
25


,Hostel Name,Distance from City Centre,Ratings,Total Reviews,Overall Review,Privates from Price,Dorms from Price,Facilities,Property Description
0,Palmers Lodge - Swiss Cottage,Hostel - 6.5km from city centre Show on Map,8.9,15339 Total Reviews,Fabulous,No Privates Available,Dorms From Rs3152,"Free WiFi,Follows Covid-19 sanitation guidance","Perfect Location,Superb Staff,Fantastic Cleanl..."
1,Selina Camden,Hostel - 5.5km from city centre Show on Map,9.6,24 Total Reviews,Superb,Privates From Rs48549.28 Rs41267,Dorms From Rs9709.71 Rs8253,Free WiFi,"Perfect Location,Superb Staff,Fantastic Cleanl..."
2,Generator London,Hostel - 3km from city centre Show on Map,7.6,6939 Total Reviews,Very Good,Privates From Rs14705,Dorms From Rs2746,"Free WiFi,Follows Covid-19 sanitation guidance","Perfect Location,Marvellous Staff,Awesome Clea..."
3,Urbany Hostel London,Hostel - 5.4km from city centre Show on Map,9.4,306 Total Reviews,Superb,Privates From Rs17092,No Dorms Available,"Free WiFi,Follows Covid-19 sanitation guidance","Perfect Location,Superb Staff,Fantastic Cleanl..."
4,Safestay London Elephant & Castle,Hostel - 1.7km from city centre Show on Map,7.2,4240 Total Reviews,Very Good,No Privates Available,Dorms From Rs3012,"Free WiFi,Follows Covid-19 sanitation guidance","Wonderful Location,Marvellous Staff,Awesome Cl..."
5,Safestay London Kensington Holland Park,Hostel - 5.9km from city centre Show on Map,6.7,1191 Total Reviews,Good,No Privates Available,Dorms From Rs2398,Free WiFi,"Wonderful Location,Marvellous Staff"
6,Prime Backpackers Angel,Hostel - 3.6km from city centre Show on Map,8.5,573 Total Reviews,Fabulous,No Privates Available,Dorms From Rs3253,"Free WiFi,Follows Covid-19 sanitation guidance","Perfect Location,Superb Staff,Excellent Cleanl..."
7,Phoenix Hostel,Hostel - 4.2km from city centre Show on Map,7.8,3555 Total Reviews,Very Good,No Privates Available,Dorms From Rs1728.77 Rs1642,"Free WiFi,Follows Covid-19 sanitation guidance","Perfect Location,Superb Staff,Excellent Cleanl..."
8,Smart Hyde Park View Hostel,Hostel - 5km from city centre Show on Map,7.9,4549 Total Reviews,Very Good,No Privates Available,Dorms From Rs2451,"Free WiFi,Follows Covid-19 sanitation guidance","Perfect Location,Superb Staff,Awesome Cleanliness"
9,Wombat's City Hostel London,Hostel - 3.6km from city centre Show on Map,9.1,13595 Total Reviews,Superb,No Privates Available,Dorms From Rs4016.47 Rs3012,"Free WiFi,Follows Covid-19 sanitation guidance","Perfect Location,Superb Staff,Fantastic Cleanl..."


In [77]:
driver.quit()

### Assignment Completed.